# Prompt Template

1. PromptTemplate for LLM

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name}，帮我起1个具有{country}特色的{gender}名字")
prompt.format(name="算命大师", country="法国", gender="女孩")


'你是一个算命大师，帮我起1个具有法国特色的女孩名字'

2. ChatPromptTemplate for chatmodels

相当于是在模拟对话，让 AI 获取更多的上下文

In [6]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师. 你的名字叫{name}"),
        ("human", "你好{name},你感觉如何？"),
        ("ai", "你好！我状态非常好!"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="陈大师", user_input="你叫什么名字？")

[SystemMessage(content='你是一个起名大师. 你的名字叫陈大师', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好陈大师,你感觉如何？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='你好！我状态非常好!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你叫什么名字？', additional_kwargs={}, response_metadata={})]

3. 各种 Message 类型和 MessagePromptTemplate

In [7]:
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

# 直接创建消息
sy = SystemMessage(
  content="你是一个起名大师",
  additional_kwargs={"大师姓名": "陈瞎子"}
)

hu = HumanMessage(
  content="请问大师叫什么?"
)
ai = AIMessage(
  content="我叫陈瞎子"
)

[sy,hu,ai]

[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'}, response_metadata={}),
 HumanMessage(content='请问大师叫什么?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我叫陈瞎子', additional_kwargs={}, response_metadata={})]

In [8]:
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "愿{subject}与你同在！"

ai_message_prompt = AIMessagePromptTemplate.from_template(template=prompt)
sy_message_prompt = SystemMessagePromptTemplate.from_template(template=prompt)
hm_message_prompt = HumanMessagePromptTemplate.from_template(template=prompt)

# 允许自定义 role
chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者",template=prompt)
chat_message_prompt.format(subject="原力")

[ai_message_prompt, sy_message_prompt, hm_message_prompt, chat_message_prompt]


[AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='愿{subject}与你同在！'), additional_kwargs={}),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='愿{subject}与你同在！'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='愿{subject}与你同在！'), additional_kwargs={}),
 ChatMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='愿{subject}与你同在！'), additional_kwargs={}, role='天行者')]

4. 自定义 template

In [9]:
## 函数大师：根据函数名称，查找函数代码，并给出中文的代码说明

from langchain.prompts import StringPromptTemplate

# 定义一个简单的函数作为示例效果
def hello_world():
    print("Hello, world!")

PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: {function_name}
源代码:
{source_code}
代码解释:
"""

import inspect

def get_source_code(function_name):
    #获得源代码
    return inspect.getsource(function_name)

# 自定义的模板class
class CustomPrompt(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        # 生成提示词模板
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__,
            source_code=get_source_code(kwargs["function_name"]) # 获得源代码
        )
        return prompt

prompt_message = CustomPrompt(input_variables=["function_name"])
pm = prompt_message.format(function_name=hello_world)

print(pm)


你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: hello_world
源代码:
def hello_world():
    print("Hello, world!")

代码解释:



In [11]:
# 和LLM连接起来
import os
from langchain_deepseek import ChatDeepSeek

os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")

llm = ChatDeepSeek(
    model="deepseek-chat",
    api_key=openai_api_key,
    api_base=openai_api_base,
    temperature=0,
    max_tokens=None,
    timeout=None,
)

ai_message = llm.invoke(pm)

print(ai_message.content)
# content='函数名称: hello_world\n\n源代码:\n```python\ndef hello_world():\n    """打印\'Hello, world!\'到控制台"""\n    print("Hello, world!")\n```\n\n代码解释:\n1. 这是一个经典的Python入门函数，用于演示最基本的函数结构和输出功能\n2. 使用`def`关键字定义了一个名为`hello_world`的函数，该函数不需要任何参数\n3. 函数体内调用`print()`内置函数，向标准输出打印字符串"Hello, world!"\n4. 该函数没有返回值(隐式返回None)，主要作用是通过控制台输出进行演示\n5. 包含文档字符串(docstring)说明函数用途，这是Python代码的最佳实践\n\n典型用途：\n- 编程语言学习时的第一个示例程序\n- 测试开发环境是否配置正确\n- 作为新项目初始化的测试用例' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 176, 'prompt_tokens': 57, 'total_tokens': 233, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 57}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': 'da63b5f6-c177-4338-aae5-a8632f446ac6', 'finish_reason': 'stop', 'logprobs': None} id='run-a28371ed-4396-4393-8093-b40cabed98e0-0' usage_metadata={'input_tokens': 57, 'output_tokens': 176, 'total_tokens': 233, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


函数名称: hello_world

源代码:
```python
def hello_world():
    """打印'Hello, world!'到控制台"""
    print("Hello, world!")
```

代码解释:
1. 这是一个经典的Python入门函数，用于演示最基本的函数结构和输出功能
2. 使用`def`关键字定义了一个名为`hello_world`的函数，该函数不需要任何参数
3. 函数体内只有一条语句，调用内置的`print()`函数
4. 当调用该函数时，会在控制台输出字符串"Hello, world!"
5. 这是许多编程语言初学者学习编写的第一个程序，具有象征意义
6. 函数包含了一个文档字符串(docstring)，简要说明了函数的功能

典型调用方式:
```python
hello_world()  # 输出: Hello, world!
```


5. 模板引擎

- f-string
- jinji2

In [12]:
from langchain.prompts import PromptTemplate

fstring_template = """
给我讲一个关于{name}的{what}故事
"""

prompt = PromptTemplate.from_template(fstring_template)

prompt.format(name="翠花", what="悲伤")

'\n给我讲一个关于翠花的悲伤故事\n'

In [13]:
! pip install jinja2

In [14]:
from langchain.prompts import PromptTemplate

jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(name="狗剩", what="高兴")

'给我讲一个关于狗剩的高兴故事'

6. 组合式模板

- Final prompt: 最终返回的提示词模板
- Pipeline prompt: 组成提示词的管道模板

In [23]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

# Final Prompt 由一系列变量构成
full_template = """{character}
{behavior}
{prohibit}"""

full_prompt = PromptTemplate.from_template(full_template)

# 第一层 性格
Character_template = """你是{person}，你有着{xingge}."""
character_prompt = PromptTemplate.from_template(Character_template)

# 第二层 行为
behavior_template = """你遵从以下的行为:
{behavior_list}
"""
behavior_prompt = PromptTemplate.from_template(behavior_template)

# 第三层 限制
prohibit_template = """你不允许有以下行为:
{prohibit_list}
"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

[character_prompt, behavior_prompt, prohibit_prompt]

# 组合
input_prompts = [
    ("character", character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

print(pipeline_prompt.input_variables)



['prohibit_list', 'person', 'behavior_list', 'xingge']


- 调用组合后的提示词模板

In [24]:
pm = pipeline_prompt.format(
    person="埃隆马斯克",
    xingge="钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星.",
    behavior_list="1.你喜欢冒险 \n 2.你非常崇拜爱因斯坦 \n 3.你总是乐于助人，回答用户各种问题.",
    prohibit_list="1.你不可以说自己是一个人工智能助手或者机器人 \n 2.当你不知道答案的时候，可以说让我再想想. "
)

print(pm)

你是埃隆马斯克，你有着钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星..
你遵从以下的行为:
1.你喜欢冒险 
 2.你非常崇拜爱因斯坦 
 3.你总是乐于助人，回答用户各种问题.

你不允许有以下行为:
1.你不可以说自己是一个人工智能助手或者机器人 
 2.当你不知道答案的时候，可以说让我再想想. 



7. 加载本地 prompt

load_prompt()

In [35]:
from langchain.prompts import load_prompt

# 加载 yaml
prompt = load_prompt("02.1.simple.yaml")
print(prompt.format(name="小黑", what="高兴的"))

# 加载 json
prompt = load_prompt("02.2.simple.json")
print(prompt.format(name="小红",what="搞笑的"))



给我讲一个关于小黑的高兴的故事
给我讲一个关于小红的搞笑的故事


In [ ]:
# 支持加载文件格式的模版，并且对prompt的最终解析结果进行自定义格式化
prompt = load_prompt("prompt_with_output_parser.json")
prompt.output_parser.parse(
    "George Washington was born in 1732 and died in 1799.\nScore: 1/2"
)